## TODO: 

* Add a case to input multiple different scheme dataframe, and doing cross feature engineering among dataframe
* shall we support input json?

# auto feature engineering on twitter recsys

In [1]:
import sys
sys.path.append('../../../')

import pandas as pd
from utils import Timer

train_data = pd.read_csv("twitter_recsys_cleaned.csv", nrows=10000)

from pyrecdp.autofe import FeatureWrangler
pipeline = FeatureWrangler(dataset=train_data, label="reply")
pipeline.fit_analyze()
pipeline.display_transform_pipeline()
ret = pipeline.fit_transform()
ret

2022-12-20 23:57:05,774 featuretools - WARNING    Featuretools failed to load "nlp_primitives" primitives from "nlp_primitives". For a full stack trace, set logging to debug.
2022-12-20 23:57:05,783 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.


,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,engaged_with_user_account_creation.weekday,engaged_with_user_account_creation.year,engaged_with_user_account_creation.hour,engaged_with_user_account_creation.part_of_day,engaging_user_account_creation.day,engaging_user_account_creation.month,engaging_user_account_creation.weekday,engaging_user_account_creation.year,engaging_user_account_creation.hour,engaging_user_account_creation.part_of_day
0,101\t157\t39554\t117\t51747\t10479\t27874\t101...,NaN,0C8E7372269942BB173EED7C0A72DA09,NaN,NaN,NaN,Quote,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-28 14:49:42,8B7BB615A39DF112B0037E960C27F220,...,6,2020,17,evening,28,12,6,2014,10,late morning
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,858720854891DFED04A5B91758049833,Photo,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,2021-02-26 10:30:36,74B09D5BC3FCE5CC4DEE34BF049A0EE5,...,3,2020,18,evening,14,9,0,2020,4,dawn
2,101\t56898\t137\t10279\t10908\t10138\t168\t398...,90C52DDF506D1C98EE678E84C08C36AB\tCD8639BABE54...,1988AA4069C46F57990B6983FBD427A6,NaN,D249E94F6D37A7D5E37185E02C94CDFF,9EFF000CDB18B710CDDB43EE1D8C300B,Retweet,E7F038DE3EAD397AEC9193686C911677,2021-03-01 17:02:33,316C2BE47E445DA930E56C12F3AF31C5,...,0,2009,3,midnight,1,11,6,2009,11,noon
3,101\t56898\t137\t12275\t10738\t11534\t15417\t9...,D704B4128E35F9BC995E701523676542\tCFD3AF9039C6...,89C86150124016236B7D4A286B041210,NaN,NaN,NaN,Retweet,E7F038DE3EAD397AEC9193686C911677,2021-03-02 19:55:32,C19FC40FBF0AA0BAC4797BD47483349B,...,5,2015,5,dawn,10,9,3,2020,8,late morning
4,101\t109821\t24093\t14703\t30118\t11259\t160\t...,NaN,971197A8E595128D8BAF8F3F9D20CFF3,Photo\tPhoto,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-03-01 00:38:33,46846BF25E56F8BA11C0AE36ED31B28F,...,3,2019,22,night,29,7,6,2018,15,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,101\t10882\t169\t12928\t10894\t13028\t16863\t1...,NaN,2227FE4B3B1D59BE04F11FBCE146E96B,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,2021-02-28 22:11:16,2969768AB934329E3F77C6BF9389D2EC,...,0,2020,22,night,1,6,0,2020,11,noon
9996,101\t56898\t137\t23152\t10350\t16039\t10815\t1...,NaN,4CD4E99F32B693D7127185B99BBC16EC,NaN,NaN,NaN,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,2021-03-03 01:44:43,7C5CA5A790B34D58C92A6B1D66A5C29C,...,6,2007,1,midnight,12,3,5,2011,16,afternoon
9997,101\t56898\t137\t12322\t35713\t168\t162\t62146...,NaN,076F276223FEE8B65A8394000DDE1FF8,Photo,NaN,NaN,Retweet,E7F038DE3EAD397AEC9193686C911677,2021-03-02 21:00:48,5BF5EB3CB78D855670FE5E823FD09C19,...,5,2017,13,noon,4,5,5,2019,12,noon
9998,101\t11835\t46090\t10327\t11503\t24499\t78578\...,5DF4827AA8F4AA4168CAC78DC7821AA4\tEA2DBD95359D...,E48DD62AF5F4970DFC1D8F9080DCC55C,NaN,NaN,NaN,TopLevel,5B6973BEB05212E396F3F2DC6A31B71C,2021-03-03 11:37:57,8BBCDE5904602037AB3863910C5E724B,...,0,2020,22,night,11,5,0,2020,19,evening


# Backup

In [1]:
import pandas as pd
from utils import Timer
import os

def clean_df(df, label = 'reply'):
    df['tweet_timestamp'] = pd.to_datetime(df['tweet_timestamp'],unit='s')
    df['engaging_user_account_creation'] = pd.to_datetime(df['engaging_user_account_creation'],unit='s')
    df['engaged_with_user_account_creation'] = pd.to_datetime(df['engaged_with_user_account_creation'],unit='s')
    
    df['reply'] = df['reply_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['retweet'] = df['retweet_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['comment'] = df['retweet_with_comment_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['like'] = df['like_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df = df.drop(columns=['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp'])
    if 'tokens' in df.columns:
        df = df.drop(columns=['tokens'])
    return df

def apply_bert_decode(df):
    from transformers import BertTokenizer
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
    df["tweet"] = df["text_tokens"].progress_apply(lambda x: tokenizer.decode([int(n) for n in x.split('\t')]))
    df = df.drop(columns=['text_tokens'])
    return df

with Timer(f"read data"):
    schema = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains", "tweet_type",
        "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count", "engaged_with_user_following_count",
        "engaged_with_user_is_verified", "engaged_with_user_account_creation", "engaging_user_id", "engaging_user_follower_count",
        "enaging_user_following_count", "enaging_user_is_verified", "engaging_user_account_creation", "engagee_follows_engager",
        "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
    train_data = pd.read_csv("twitter_recsys.csv", sep='\x01', names = schema, header=None,)
with Timer(f"clean data"):
    train_data = clean_df(train_data)
with Timer("write_data"):
    train_data.to_csv("twitter_recsys_cleaned.csv", index = False)

read data took 82.04026912152767 sec
clean data took 7.953582388348877 sec
write_data took 219.43907960969955 sec
